In [ ]:
%pip install -r requirements.txt

In [ ]:
from dataset import QuickDrawDataset
from sketch_tokenizers import DeltaPenPositionTokenizer
from models import SketchTransformerConditional
from runner import SketchTrainer, sample, device
import torch

if device == "cuda":
    # Empty the cache to free up GPU memory
    torch.cuda.empty_cache()

In [ ]:
label_names = ["bird", "crab", "guitar"]
dataset = QuickDrawDataset(label_names=label_names)
tokenizer = DeltaPenPositionTokenizer(bins=32)

model = SketchTransformerConditional(
    vocab_size=len(tokenizer.vocab),
    d_model=512,
    nhead=8,
    num_layers=8,
    max_len=200,
    num_classes=len(label_names),
)

training_config = {
    "batch_size": 128,
    "num_epochs": 15,
    "learning_rate": 1e-4,
    "log_dir": "logs/sketch_transformer_experiment_2",
    "splits": [0.85, 0.1, 0.05],
}

trainer = SketchTrainer(model, dataset, tokenizer, training_config)

In [ ]:
trainer.train_mixed(training_config["num_epochs"])

In [ ]:
from IPython.display import HTML, display
from prepare_data import stroke_to_bezier_single, clean_svg

generations_inline = ""
generations = []

for j, label_name in enumerate(label_names):
    for i in range(5):
        generated = sample(
            model=trainer.model,
            start_tokens=[trainer.tokenizer.vocab["START"]],
            temperature=0.8,
            top_k=20,
            top_p=0.7,
            greedy=False,
            eos_id=trainer.tokenizer.vocab["END"],
            class_label=j,
        )
        decoded_sketch = tokenizer.decode(generated, stroke_width=0.3)
        decoded_sketch = stroke_to_bezier_single(decoded_sketch)
        decoded_sketch = clean_svg(decoded_sketch)

        generations_inline += f'<div style="display:inline-block; width: 150px; background-color: white; margin-right:10px;"><b>Generated {label_name}</b><br>{decoded_sketch}</div>'
        generations.append((generated, decoded_sketch))

display(HTML(generations_inline))